## Initial configuration

In [4]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
import cv2
import gc
from tqdm import tqdm,trange

import torch
from torchsummary import summary
from PIL import Image
from torch.utils.data import Dataset, DataLoader

import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.optim as optim
import matplotlib.pyplot as plt

import urllib.request
import zipfile
import json

In [5]:
#Usefull for better outputs visualization
PURPLE = "\033[95m"
CYAN = "\033[96m"
DARKCYAN = "\033[36m"
BLUE = "\033[94m"
GREEN = "\033[92m"
YELLOW = "\033[93m"
RED = "\033[91m"
BOLD = "\033[1m"
UNDERLINE = "\033[4m"
END = "\033[0m"

### Project settings

In [6]:
data_dir_unsplitted = "dataset/unsplitted"
save_dir_splitted = "dataset/splitted"
data_dir = "dataset/splitted"
models_dir ="models/" #the resuls will be saved here

os.makedirs(models_dir,exist_ok=True)

grayscale= True
resize_to= (347,260)
fft = True

learning_rate_e = 0.001
learning_rate_d = 0.001
#learning_rate_c = 0.00001
num_epochs = 50


#@DEPRECATED VARIABLE
ALREADY_SPLITTED = True #-> we will use only "LOAD_DATASET"


LOAD_DATASET = True #if true "ALREADY_SPLITTED" is ignored

#to be implemented:

DATASET_ALREADY_DOWNLOADED = True 
DATASET_ALREADY_UNZIPPED = True

LOAD_MODEL = False 
MODEL_ALREADY_DOWNLOADED = True 

## Dataset preparation

### Dataset splitting and data augmentation

In [7]:
classes = ["CLL", "FL", "MCL"]
id_to_name = {0:"CLL",1:"FL",2:"MCL"}

if(not LOAD_DATASET):
    data = []
    labels = []
    for i, cl in enumerate(classes):
        path = os.path.join(data_dir_unsplitted, cl)
        images = os.listdir(path)
        for img in tqdm(images,desc=cl):
            img_path = os.path.join(path, img)
            img = cv2.imread(img_path).astype(np.uint8)
            
            data.append(img)
            labels.append(i) 


    data = np.array(data)
    labels = np.array(labels)


    train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=42)


    train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.125, random_state=42)

    print("Train samples:", len(train_data))
    print("Test samples:", len(test_data))
    print("Validation samples:", len(val_data))

### Saving the splitted dataset

In [8]:
#SAVING THE DATASET
if(not LOAD_DATASET):
    #TRAIN
    os.makedirs(os.path.join(save_dir_splitted,"train","images"),exist_ok=True)
    os.makedirs(os.path.join(save_dir_splitted,"train","labels"),exist_ok=True)

     
    labels_train_augmented={}
    for i, image in enumerate(tqdm(train_data,desc = "Train")):                       #images
        file_name = str(i)+"_train"+".png"
        
        path = os.path.join(save_dir_splitted,"train","images",file_name)
        labels_train_augmented[file_name] = int(train_labels[i])
        cv2.imwrite(path,image.astype(np.float64))
        
        #flipping
        flipped_img = np.fliplr(image)
        
        file_name = str(i)+"_flippend_train"+".png"
        path = os.path.join(save_dir_splitted,"train","images",file_name)
        labels_train_augmented[file_name] = int(train_labels[i])
        cv2.imwrite(path,flipped_img.astype(np.float64))
        
        #gaussian noise
        noise = np.random.normal(0, 5, img.shape).astype(np.uint8)
        noisy_image = image.astype(np.uint8) + noise
        
        file_name = str(i)+"_noisy_train"+".png"
        path = os.path.join(save_dir_splitted,"train","images",file_name)
        labels_train_augmented[file_name] = int(train_labels[i])
        cv2.imwrite(path,noisy_image.astype(np.float64))
    
    with open(os.path.join(save_dir_splitted,"train","labels","train.json"), 'w') as f:
        json.dump(labels_train_augmented, f)#labels

    #TEST
    os.makedirs(os.path.join(save_dir_splitted,"test","images"),exist_ok=True)
    os.makedirs(os.path.join(save_dir_splitted,"test","labels"),exist_ok=True)

    
    
    test_labels_dict = {}
    for i, image in enumerate(tqdm(test_data,desc = "Test")):                      #images
        file_name = str(i)+"_test"+".png"
        path = os.path.join(save_dir_splitted,"test","images",file_name)
        
        cv2.imwrite(path,image.astype(np.float64))
        test_labels_dict[file_name] = int(test_labels[i]) 

    with open(os.path.join(save_dir_splitted,"test","labels","test.json"), 'w') as f:
        json.dump(test_labels_dict, f)#labels
        
    #VALIDATION
    os.makedirs(os.path.join(save_dir_splitted,"val","images"),exist_ok=True)
    os.makedirs(os.path.join(save_dir_splitted,"val","labels"),exist_ok=True)

    
    
    val_labels_dict = {}
    for i, image in enumerate(tqdm(val_data,desc = "Validation")):               #images
        file_name = str(i)+"_val"+".png"
        path = os.path.join(save_dir_splitted,"val","images",file_name)
        
        cv2.imwrite(path,image.astype(np.float64))
        val_labels_dict[file_name] = int(val_labels[i])
        
    with open(os.path.join(save_dir_splitted,"val","labels","val.json"), 'w') as f:
        json.dump(val_labels_dict, f)#labels

### Dataloaders definition

Notice that the labels are saved into a numpy array, while the images are read directly from the folder

In [66]:
class HDAdataset(Dataset):
    def __init__(self, root_dir, labels, train = False, transform=None, grayscale= False, resize_to= None, fft = False):
        self.root_dir = root_dir
        self.labels = labels
        self.transform = transform
        self.train = train
        self.grayscale=grayscale
        self.fft = fft
        self.resize_to = resize_to

    def __len__(self):
        return len(os.listdir(self.root_dir))

    def __getitem__(self, idx):
        img_name = os.listdir(self.root_dir)[idx]
        img_path = os.path.join(self.root_dir, img_name)
        
        if self.grayscale or self.fft:
            image = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE)
        
        if self.train:
            image = image.astype(np.uint8) + np.random.normal(0, 2, image.shape).astype(np.uint8)
        
        if self.resize_to is not None:
            image = cv2.resize(image, resize_to, interpolation= cv2.INTER_LINEAR)
            
        if self.fft:
            image_fft = np.fft.fft2(image)
            fshift = np.fft.fftshift(image_fft)
            magnitude_spectrum = 20*np.log(np.abs(fshift))
            image_fft = magnitude_spectrum
        

        
        #labels smoothing
        label = 0.1*np.ones(3)
        label[self.labels[img_name]] = 0.8
        
        if self.transform:
            image = self.transform((image/255.).astype(np.float32))
            image_fft = self.transform((image_fft/255.).astype(np.float32))

        return image,image_fft, label

In [67]:
if(LOAD_DATASET and not DATASET_ALREADY_DOWNLOADED):
    print("download the dataset")
    urllib.request.urlretrieve("", "dataset.zip")
    
if(LOAD_DATASET and not DATASET_ALREADY_UNZIPPED):
    with zipfile.ZipFile("dataset.zip", 'r') as zip_ref:
        zip_ref.extractall("./dataset/")


#convert to tensor each image
transform = transforms.Compose([transforms.ToTensor()])    

#TRAIN DATASET AND DATALOADER
train_dir = os.path.join(data_dir,"train","images")#images
train_labels_dir = os.path.join(data_dir,"train","labels","train.json")
with open(train_labels_dir) as f:
    train_labels = json.load(f)#labels

train_dataset = HDAdataset(train_dir,train_labels,train = False,grayscale = grayscale, resize_to= resize_to,
                           transform=transform,fft = fft)
train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True,drop_last=True)

#TEST DATASET AND DATALOADER
test_dir = os.path.join(data_dir,"test","images")#images
test_labels_dir = os.path.join(data_dir,"test","labels","test.json")
with open(test_labels_dir) as f:
    test_labels = json.load(f)#labels

test_dataset = HDAdataset(test_dir,test_labels,train = False,grayscale = grayscale, resize_to= resize_to,
                           transform=transform,fft = fft)
test_dataloader = DataLoader(test_dataset, batch_size=10, shuffle=False)

#VALIDATION DATASET AND DATALOADER
val_dir = os.path.join(data_dir,"val","images")#images

val_labels_dir = os.path.join(data_dir,"val","labels","val.json")
with open(val_labels_dir) as f:
    val_labels = json.load(f)#labels

val_dataset = HDAdataset(val_dir,val_labels,train = False,grayscale = grayscale, resize_to= resize_to,
                           transform=transform,fft = fft)
val_dataloader = DataLoader(val_dataset, batch_size=10, shuffle=False)

## Neural network

In [68]:
torch.cuda.empty_cache()
gc.collect()

1401

### Models definition

In [69]:
class Rgb(nn.Module):
    def __init__(self):
        super(Rgb, self).__init__()
        
        self.features_extraction = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(3,3), stride=(2,2), padding=(1,1)),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(),
            nn.Dropout(p=0.1),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3,3), stride=(3,3), padding=(1,1)),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(),
            nn.Dropout(p=0.1),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3), stride=(3,3), padding=(1,1)),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(),
            nn.Flatten(),
            nn.LeakyReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(256,150),
            nn.BatchNorm1d(150),
            nn.LeakyReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(150,50),
            nn.LeakyReLU()
        )

    def forward(self, x):
        x = self.features_extraction(x)
        return x
    
class Fft(nn.Module):
    def __init__(self):
        super(Fft, self).__init__()
        
        self.features_extraction = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(3,3), stride=(2,2), padding=(1,1)),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(),
            nn.Dropout(p=0.1),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3,3), stride=(3,3), padding=(1,1)),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(),
            nn.Dropout(p=0.1),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3), stride=(3,3), padding=(1,1)),
            nn.MaxPool2d(2),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(),
            nn.Flatten(),
            nn.LeakyReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(256,150),
            nn.BatchNorm1d(150),
            nn.LeakyReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(150,50),
            nn.LeakyReLU()
        )

    def forward(self, x):
        x = self.features_extraction(x)
        return x
    
    
    
class Classifier(nn.Module):
    def __init__(self,rgb,fft):
        super(Classifier, self).__init__()

        self.rgb = rgb
        self.fft = fft
        self.classifier = nn.Sequential(
            nn.Linear(100,20),
            nn.BatchNorm1d(20),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(20,3),
            nn.Softmax(dim=1)
        )

    def forward(self, x,x_fft):
        x1 = self.rgb(x)
        x2 = self.fft(x_fft)
        x = torch.cat((x1,x2), 1)
        x = self.classifier(x)

        return x


In [70]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(BOLD+"You are working on: "+END+str(device))

You are working on: cuda


### Loading of the models

In [71]:
if(LOAD_MODEL and not MODEL_ALREADY_DOWNLOADED):
    print("download the model")

In [72]:
if(not LOAD_MODEL):
    rgb = Rgb().to(device)
    fft = Fft().to(device)
    classifier = Classifier(rgb,fft).to(device)

In [75]:
summary(classifier, [(1, 260, 347),(1, 260, 347)],batch_size=3)
# summary(decoder, (1,60),batch_size=3)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [3, 16, 130, 174]             160
         MaxPool2d-2            [3, 16, 65, 87]               0
       BatchNorm2d-3            [3, 16, 65, 87]              32
         LeakyReLU-4            [3, 16, 65, 87]               0
           Dropout-5            [3, 16, 65, 87]               0
            Conv2d-6            [3, 32, 22, 29]           4,640
         MaxPool2d-7            [3, 32, 11, 14]               0
       BatchNorm2d-8            [3, 32, 11, 14]              64
         LeakyReLU-9            [3, 32, 11, 14]               0
          Dropout-10            [3, 32, 11, 14]               0
           Conv2d-11              [3, 64, 4, 5]          18,496
        MaxPool2d-12              [3, 64, 2, 2]               0
      BatchNorm2d-13              [3, 64, 2, 2]             128
        LeakyReLU-14              [3, 6

### Training 

In [80]:
def test(dataloader,name):
    
    losses = []

    count_correct = 0
    count_input = 0

    classifier.eval()
    
    for data in tqdm(dataloader,desc = name +" set"):

        img,img_fft, labels = data
        img = img.to(device)
        labels = labels.to(device)
        img_fft = img_fft.to(device)


        pred_labels = classifier(img,img_fft)


        classifier_loss = criterion_classifier(pred_labels, labels)
        
        losses.append(classifier_loss.detach().cpu().numpy())

        count_input+=pred_labels.detach().cpu().numpy().shape[0]
        count_correct+=np.sum((labels.to("cpu").detach().numpy().argmax(axis=1)==pred_labels.to("cpu").detach().numpy().argmax(axis=1)))
        del img
        del labels
        torch.cuda.empty_cache()
        gc.collect();

    # Print loss at end of each epoch
    accuracy= (count_correct/count_input)*100.
    print(name,' Classifier Loss: %.4f, Accuracy: %.4f in %d input samples\n'
      % (np.mean(losses),accuracy,count_input))
    
    return np.mean(losses),accuracy

In [83]:
criterion_classifier = nn.CrossEntropyLoss()

def train(train_dataloader,val_dataloader):
    optimizer = optim.Adam(classifier.parameters(), lr=learning_rate_e,weight_decay=1e-2)


    
    
    best_epoch = 0 #(based on the validation accuracy)
    best_accuracy = -np.inf
    
    #for plotting
    train_loss_trend = []
    train_accuracy_trend =[]
    val_loss_trend = []
    val_accuracy_trend = []

    
    for epoch in range(num_epochs):

        ######training#####
        losses = []
        count_correct = 0
        count_input = 0

        classifier.train()
            

        #train classifier for one epoch    
        for data in tqdm(train_dataloader,desc = "Train classifier Epoch [%d/%d]"%(epoch+1,num_epochs)):

            img,img_fft, labels = data
            img = img.to(device)
            labels = labels.to(device)
            img_fft = img_fft.to(device)
            optimizer.zero_grad()
            

            pred_labels= classifier(img,img_fft)
            

            classifier_loss = criterion_classifier(pred_labels, labels)

            classifier_loss.backward()
            
            losses.append(classifier_loss.detach().cpu().numpy())

            count_input+=pred_labels.detach().cpu().numpy().shape[0]
            count_correct+=np.sum((labels.to("cpu").detach().numpy().argmax(axis=1)==pred_labels.to("cpu").detach().numpy().argmax(axis=1)))
            del img
            del labels
            torch.cuda.empty_cache()
            gc.collect();
            
        accuracy = (count_correct/count_input)*100.
        
        print('TRAIN: Epoch [%d/%d], Classifier Loss: %.4f, Accuracy: %.4f in %d input samples \n'
              % (epoch+1, num_epochs,np.mean(losses),accuracy,count_input))
        
        train_loss_trend.append(np.mean(losses))
        train_accuracy_trend.append(accuracy)
        
        ######validation######
        l,acc = test(val_dataloader,"VALIDATION")
        
        val_loss_trend.append(l)
        val_accuracy_trend.append(acc)
        
        if acc > best_accuracy:
            print(GREEN+"Saved Model: "+END+UNDERLINE+"best validation accuracy reached\n\n"+END)
            best_epoch = epoch+1
            torch.save(classifier.state_dict(), os.path.join(models_dir,"best_classifier.pt"))
            best_accuracy = acc

    return np.array([[train_loss_trend,train_accuracy_trend],
            [val_loss_trend,val_accuracy_trend],best_epoch],dtype=np.object_)

In [ ]:
if(not LOAD_MODEL):
    results = train(train_dataloader,val_dataloader)
    np.save(os.path.join(models_dir,"results.npy"),results)

Train classifier Epoch [1/50]: 100%|███████████████████████████████████████████████████| 78/78 [00:38<00:00,  2.01it/s]


TRAIN: Epoch [1/50], Classifier Loss: 1.1036, Accuracy: 32.4359 in 780 input samples 



VALIDATION set: 100%|████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  2.23it/s]


VALIDATION  Classifier Loss: 1.1091, Accuracy: 26.3158 in 38 input samples

Saved Model: best validation accuracy reached




Train classifier Epoch [2/50]: 100%|███████████████████████████████████████████████████| 78/78 [00:39<00:00,  1.98it/s]


TRAIN: Epoch [2/50], Classifier Loss: 1.1030, Accuracy: 34.3590 in 780 input samples 



VALIDATION set: 100%|████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  2.21it/s]


VALIDATION  Classifier Loss: 1.1094, Accuracy: 31.5789 in 38 input samples

Saved Model: best validation accuracy reached




Train classifier Epoch [3/50]:  31%|███████████████▋                                   | 24/78 [00:12<00:27,  1.98it/s]

In [ ]:
#always load the best model after the training
encoder =  Encoder().to(device)
decoder = Decoder().to(device)
encoder.load_state_dict(torch.load(os.path.join(models_dir,"best_e.pt")))
encoder.eval();
decoder.load_state_dict(torch.load(os.path.join(models_dir,"best_d.pt")))
decoder.eval();

### Test

Notice that the value of the losses and the accuraciesfor the train set are computed during the training, so they are not very accurate.

In [ ]:
# train_dataset_no_noise = HDAdataset(train_dir,train_labels,train = False, transform=transform)
# train_dataloader_no_noise = DataLoader(train_dataset_no_noise, batch_size=64, shuffle=False,drop_last=False)

In [ ]:
test(train_dataloader,"TRAIN")
test(val_dataloader,"VALIDATION")
test(test_dataloader,"TEST");